In [24]:
import numpy as np
import pandas as pd

 # 유성우 예측

 ### tl;dr

 - 혜성이 지나간 자리에 유성체 stream이 발생하고 이 것이 지구의 공전 궤도와 만나면 유성우가 발생
 - 유성우는 관측자의 위치, 관측 시기, 관측 날짜의 달의 위상 등에 따라 관측 여부가 정해짐
 - 그럼 내가 있는 도시에서 언제 유성우를 볼 수 있는지 예측할 수 있을까?

 ### 유성우 예측을 위해 필요한 정보

 - https://github.com/drguthals/learnwithdrg/tree/main/OverTheMoon/meteor-showers/data
 - 유성우가 발생하는 기간 (4개의 혜성으로부터 발생하는 주요 5대 유성우): meteorshowers.csv
 - 유성우를 관측할 수 있는 위도: constellations.csv
 - 달의 위상: moonphases.csv
 - 도시 위도: cities.csv

In [25]:
meteor_showers = pd.read_csv('data/meteorshowers.csv') 
moon_phases = pd.read_csv('data/moonphases.csv')
constellations = pd.read_csv('data/constellations.csv')
cities = pd.read_csv('data/cities.csv')

In [26]:
change_meteor_shower = {'name': 'Chang\'e', 'radiant': 'Draco', 'bestmonth': 'october', 'startmonth': 'october', 'startday': 1, 'enddya': 31, 'hemisphere': 'northern', 'preferredhemisphere': 'northern'}
meteor_showers = pd.concat([meteor_showers, pd.DataFrame([change_meteor_shower])], ignore_index=True)

In [27]:
draco_constellation = {'constellation': 'Draco', 'bestmonth': 'july', 'latitudestart': 90, 'latitudeend': -15, 'besttime': 2100, 'hemisphere': 'northern'}
constellations = pd.concat([constellations, pd.DataFrame([draco_constellation])], ignore_index=True)

In [28]:
meteor_showers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 6 non-null      object 
 1   radiant              6 non-null      object 
 2   bestmonth            6 non-null      object 
 3   startmonth           6 non-null      object 
 4   startday             6 non-null      int64  
 5   endmonth             5 non-null      object 
 6   endday               5 non-null      float64
 7   hemisphere           6 non-null      object 
 8   preferredhemisphere  6 non-null      object 
 9   enddya               1 non-null      float64
dtypes: float64(2), int64(1), object(7)
memory usage: 612.0+ bytes


In [29]:
moon_phases.head()

,month,day,moonphase,specialevent
0,january,1,NaN,NaN
1,january,2,first quarter,NaN
2,january,3,NaN,NaN
3,january,4,NaN,NaN
4,january,5,NaN,NaN


In [30]:
moon_phases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   month         366 non-null    object
 1   day           366 non-null    int64 
 2   moonphase     50 non-null     object
 3   specialevent  10 non-null     object
dtypes: int64(1), object(3)
memory usage: 11.6+ KB


In [31]:
constellations.head()

,constellation,bestmonth,latitudestart,latitudeend,besttime,hemisphere
0,Lyra,august,90,-40,21:00,northern
1,Aquarius,october,65,-90,21:00,southern
2,Orion,january,85,-75,21:00,northern
3,Perseus,december,90,-35,21:00,northern
4,Leo,april,90,65,21:00,northern


In [32]:
constellations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   constellation  6 non-null      object
 1   bestmonth      6 non-null      object
 2   latitudestart  6 non-null      int64 
 3   latitudeend    6 non-null      int64 
 4   besttime       6 non-null      object
 5   hemisphere     6 non-null      object
dtypes: int64(2), object(4)
memory usage: 420.0+ bytes


In [33]:
cities.head()

,city,latitude,country
0,Abu Dhabi,24.47,United Arab Emirates
1,Abuja,9.07,Nigeria
2,Accra,5.55,Ghana
3,Adamstown,-25.07,Pitcairn Islands
4,Addis Ababa,9.02,Ethiopia


In [34]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   city      256 non-null    object 
 1   latitude  256 non-null    float64
 2   country   256 non-null    object 
dtypes: float64(1), object(2)
memory usage: 6.1+ KB


### Data 준비

In [35]:
months = {'january':1, 'february':2, 'march':3, 'april':4, 'may':5, 'june':6, 'july':7, 'august':8, 'september':9, 'october':10, 'november':11, 'december':12}
meteor_showers.bestmonth = meteor_showers.bestmonth.map(months)
meteor_showers.startmonth = meteor_showers.startmonth.map(months)
meteor_showers.endmonth = meteor_showers.endmonth.map(months)
moon_phases.month = moon_phases.month.map(months)
constellations.bestmonth = constellations.bestmonth.map(months)

In [36]:
meteor_showers.head()

,name,radiant,bestmonth,startmonth,startday,endmonth,endday,hemisphere,preferredhemisphere,enddya
0,Lyrids,Lyra,4,4,21,4.0,22.0,northern,northern,NaN
1,Eta Aquarids,Aquarius,5,4,19,5.0,28.0,"northern, southern",southern,NaN
2,Orionids,Orion,10,10,2,11.0,7.0,"northern, southern","northern, southern",NaN
3,Perseids,Perseus,8,7,14,8.0,24.0,northern,northern,NaN
4,Leonids,Leo,11,11,6,11.0,30.0,"northern, southern","northern, southern",NaN


In [37]:
meteor_showers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 6 non-null      object 
 1   radiant              6 non-null      object 
 2   bestmonth            6 non-null      int64  
 3   startmonth           6 non-null      int64  
 4   startday             6 non-null      int64  
 5   endmonth             5 non-null      float64
 6   endday               5 non-null      float64
 7   hemisphere           6 non-null      object 
 8   preferredhemisphere  6 non-null      object 
 9   enddya               1 non-null      float64
dtypes: float64(3), int64(3), object(4)
memory usage: 612.0+ bytes


In [38]:
meteor_showers['startdate'] = pd.to_datetime(2020*10000+meteor_showers.startmonth*100+meteor_showers.startday,format='%Y%m%d')
meteor_showers['enddate'] = pd.to_datetime(2020*10000+meteor_showers.endmonth*100+meteor_showers.endday,format='%Y%m%d')

In [39]:
moon_phases['date'] = pd.to_datetime(2020*10000+moon_phases.month*100+moon_phases.day,format='%Y%m%d')

In [40]:
hemispheres = {'northern':0, 'southern':1, 'northern, southern':3}
meteor_showers.hemisphere = meteor_showers.hemisphere.map(hemispheres)
constellations.hemisphere = constellations.hemisphere.map(hemispheres)

In [41]:
phases = {'new moon':0,'third quarter':0.5, 'first quarter':0.5,'full moon':1.0}
moon_phases['percentage'] = moon_phases.moonphase.map(phases)
moon_phases.head()

,month,day,moonphase,specialevent,date,percentage
0,1,1,NaN,NaN,2020-01-01,NaN
1,1,2,first quarter,NaN,2020-01-02,0.5
2,1,3,NaN,NaN,2020-01-03,NaN
3,1,4,NaN,NaN,2020-01-04,NaN
4,1,5,NaN,NaN,2020-01-05,NaN


In [45]:
meteor_showers = meteor_showers.drop(['startmonth', 'startday', 'endmonth', 'endday', 'hemisphere'], axis=1)
moon_phases = moon_phases.drop(['month','day','moonphase','specialevent'], axis=1)
constellations = constellations.drop(['besttime'], axis=1)

In [48]:
moon_phases

,date,percentage
0,2020-01-01,0.0
1,2020-01-02,0.5
2,2020-01-03,0.5
3,2020-01-04,0.5
4,2020-01-05,0.5
...,...,...
361,2020-12-27,0.5
362,2020-12-28,0.5
363,2020-12-29,1.0
364,2020-12-30,1.0


In [47]:
lastPhase = 0

for index, row in moon_phases.iterrows():
    if pd.isnull(row['percentage']):
        moon_phases.at[index,'percentage'] = lastPhase
    else:
        lastPhase = row['percentage']
        
moon_phases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        366 non-null    datetime64[ns]
 1   percentage  366 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.8 KB


In [49]:
meteor_showers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   name                 6 non-null      object        
 1   radiant              6 non-null      object        
 2   bestmonth            6 non-null      int64         
 3   preferredhemisphere  6 non-null      object        
 4   enddya               1 non-null      float64       
 5   startdate            6 non-null      datetime64[ns]
 6   enddate              5 non-null      datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(3)
memory usage: 468.0+ bytes


In [50]:
moon_phases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        366 non-null    datetime64[ns]
 1   percentage  366 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.8 KB


In [51]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   city      256 non-null    object 
 1   latitude  256 non-null    float64
 2   country   256 non-null    object 
dtypes: float64(1), object(2)
memory usage: 6.1+ KB


In [52]:
constellations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   constellation  6 non-null      object
 1   bestmonth      6 non-null      int64 
 2   latitudestart  6 non-null      int64 
 3   latitudeend    6 non-null      int64 
 4   hemisphere     6 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 372.0+ bytes


In [53]:
if 'San Diego' in cities.values:
    print('here')
else:
    print('not here')

not here


In [54]:
def predict_best_meteor_shower_viewing(city):
    # Create an empty string to return the message back to the user
    meteor_shower_string = ""

    if city not in cities.values:
        meteor_shower_string = "Unfortunately, " + city + " isn't available for a prediction at this time."
        return meteor_shower_string

    # Get the latitude of the city from the cities dataframe
    latitude = cities.loc[cities['city'] == city, 'latitude'].iloc[0]

    # Get the list of constellations that are viewable from that latitude
    constellation_list = constellations.loc[(constellations['latitudestart'] >= latitude) & (constellations['latitudeend'] <= latitude), 'constellation'].tolist()

    # If no constrllations are viewable, let the user know
    if not constellation_list:
        meteor_shower_string = "Unfortunately, there are no meteor showers viewable from "+ city + "."

        return meteor_shower_string

    meteor_shower_string = "In " + city + " you can see the following meteor showers:\n"
    
    # Iterate through each constellation that is viewable from the city
    for constellation in constellation_list:
        # Find the meteor shower that is nearest that constellation
        meteor_shower = meteor_showers.loc[meteor_showers['radiant'] == constellation, 'name'].iloc[0]

        # Find the start and end dates for that meteor shower
        meteor_shower_startdate = meteor_showers.loc[meteor_showers['radiant'] == constellation, 'startdate'].iloc[0]
        meteor_shower_enddate = meteor_showers.loc[meteor_showers['radiant'] == constellation, 'enddate'].iloc[0]

        # Find the moon phases for each date within the viewable timeframe of that meteor shower
        moon_phases_list = moon_phases.loc[(moon_phases['date'] >= meteor_shower_startdate) & (moon_phases['date'] <= meteor_shower_enddate)]

        if meteor_shower == 'Chang\'e':
            # For the film meteor shower, find the date where the moon is the most visible
            best_moon_date = moon_phases_list.loc[moon_phases_list['percentage'].idxmax()]['date']

            # Add that date to the string to report back to the user
            meteor_shower_string += "Though the moon will be bright, the " + meteor_shower + " is best seen if you look towards the " + constellation + " constellation on " +  best_moon_date.to_pydatetime().strftime("%B %d, %Y") + ".\n"
        else:
            # Find the first date where the moon is the least visible
            best_moon_date = moon_phases_list.loc[moon_phases_list['percentage'].idxmin()]['date']

            # Add that date to the string to report back to the user
            meteor_shower_string += meteor_shower + " is best seen if you look towards the " + constellation + " constellation on " +  best_moon_date.to_pydatetime().strftime("%B %d, %Y") + ".\n"
    
    return meteor_shower_string

In [55]:
print(predict_best_meteor_shower_viewing('Beijing'))

ValueError: attempt to get argmax of an empty sequence